In [1]:
import gzip
import ast
import pandas as pd
from textblob import TextBlob
from datetime import datetime

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
with gzip.open('steam_games.json.gz', 'rb') as f:
    steam_raw_df = pd.read_json(f, lines=True)

In [4]:
def alt_load(route):
    df=[]
    with gzip.open(route, 'rb') as f:
        file = f.read().decode('utf-8')
        for line in file.split('\n'):
            if line:
                df.append(ast.literal_eval(line))
    return pd.DataFrame(df)

In [5]:
reviews_raw_df = alt_load('user_reviews.json.gz')   

In [6]:
items_raw_df = alt_load('users_items.json.gz')

In [7]:
steam_df = steam_raw_df.dropna(how='all')

In [8]:
reviews_df = reviews_raw_df['reviews'].to_frame()

In [9]:
reviews_df = reviews_df.explode('reviews').apply(pd.Series)

In [10]:
reviews_df = pd.json_normalize(reviews_df['reviews'])

In [11]:
reviews_df = reviews_df.drop(columns=['funny','last_edited','helpful'])

In [12]:
reviews_df.rename(columns={'posted':'date_str'},inplace=True)

In [13]:
reviews_df.dropna(how='all',inplace=True)

In [14]:
def analyze_sentiment(review):
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < 0:
        return 0  
    elif polarity == 0:
        return 1  
    else:
        return 2  

In [15]:
reviews_df['sentiment_analysis'] = reviews_df['review'].apply(lambda x: analyze_sentiment(str(x)))

In [16]:
reviews_df['recommend'] = reviews_df['recommend'].astype(int)

In [17]:
reviews_df['date_str'] = reviews_df['date_str'].str.replace('Posted ', '')

In [18]:
reviews_df['date'] = pd.to_datetime(reviews_df['date_str'], format='%B %d, %Y.', errors='coerce')


In [19]:
reviews_df['date'] = reviews_df['date'].fillna(pd.to_datetime(reviews_df[reviews_df['date'].isnull()]['date_str'] + str(datetime.now().year),format='%B %d.%Y'))


In [20]:
reviews_df.drop(columns={'date_str','review'},inplace=True)

In [21]:
reviews_df.reset_index(inplace=True)

In [22]:
reviews_df.drop(columns={'index'},inplace=True)

In [23]:
steam_df.drop(columns={'publisher','title','url','reviews_url','price','early_access','developer'},inplace=True)

In [24]:
steam_df.set_index('id', inplace=True)


In [25]:
steam_df.dropna(how='all',inplace=True)

In [26]:
steam_df['date'] = pd.to_datetime(steam_df['release_date'],format='mixed', errors='coerce')

In [27]:
steam_df.drop(columns=['release_date'],inplace=True)

In [28]:
steam_df.rename(columns={'app_name':'name'},inplace=True)

In [29]:
steam_df[['tags','genres','specs']]=steam_df[['tags','genres','specs']].applymap(lambda x: x if isinstance(x,list) else [])

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_496\2103811663.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  steam_df[['tags','genres','specs']]=steam_df[['tags','genres','specs']].applymap(lambda x: x if isinstance(x,list) else [])


In [30]:
steam_df['genres']=steam_df.apply(lambda row: row['genres'] + row['tags'] + row['specs'], axis=1)

In [31]:
steam_df['genres']=steam_df['genres'].apply(lambda x: list(set(x)))

In [32]:
steam_df.drop(columns={'tags','specs'},inplace=True)

In [33]:
items_df = items_raw_df.drop(columns={'items_count','user_url','steam_id'})

In [34]:
items_df = items_df.explode('items').apply(pd.Series)

In [35]:
items_df.reset_index(inplace=True)

In [36]:
items_df.drop(columns=['index'],inplace=True)

In [37]:
items_df = pd.json_normalize(items_df['items']).join(items_df['user_id'])

In [38]:
items_df.dropna(subset=['item_id'],inplace=True)

In [39]:
items_df['item_id'] = items_df['item_id'].astype(int)

In [40]:
items_df.drop(columns={'playtime_2weeks'},inplace=True)

In [41]:
items_df.rename(columns={'playtime_forever':'playtime'},inplace=True)

In [42]:
steam_df.to_parquet('steam_games.parquet')
reviews_df.to_parquet('user_reviews.parquet')
items_df.to_parquet('users_items.parquet')